In [112]:
import pandas as pd
import numpy as np

In [113]:
import os

cwd = os.getcwd()
col_path = os.path.join(cwd, "raw/cost_of_living_cities.csv")
country_codes_path = os.path.join(cwd,"raw/ISO-3166-Countries-with-Regional-Code.csv")

In [114]:
col = pd.read_csv(col_path)
print(col.head())
len(col)


   Rank                   City  Cost of Living Index  Rent Index  \
0     1      Hamilton, Bermuda                 140.4        93.6   
1     2     Basel, Switzerland                 130.0        46.4   
2     3    Zurich, Switzerland                 128.5        69.1   
3     4  Lausanne, Switzerland                 120.6        53.4   
4     5       Zug, Switzerland                 119.1        66.7   

   Cost of Living Plus Rent Index  Groceries Index  Restaurant Price Index  \
0                           118.1            144.1                   141.9   
1                            90.2            137.4                   121.7   
2                           100.2            120.4                   129.7   
3                            88.6            116.1                   114.7   
4                            94.1            112.6                   120.7   

   Local Purchasing Power Index           Year  
0                          74.3  2023 Mid-Year  
1                       

11793

In [115]:
# separate city and country into two columns
city_country = col['City'].str.split(', ', expand=True)

#drop old name column
col.drop(columns=["City"], inplace=True)

col['Country'] = city_country[2].fillna(city_country[1])
col['Country'] = col['Country'].str.replace(r'\s*\(.*?\)', '', regex=True)
col['Country'] = col['Country'].str.replace(r'\bAnd\b', 'and', regex=True)
col['Country'] = col['Country'].str.replace(r'\bOf\b', 'of', regex=True)

col['City'] = city_country[0]

#convert mid-years to June of the same year
def convert_to_date(year_str):
    if 'Mid-Year' in year_str:
        return pd.to_datetime(year_str.replace('Mid-Year', ' June'), format='%Y %B')
    else:
        return pd.to_datetime(year_str, format='%Y')

col['Year'] = col['Year'].apply(convert_to_date)

In [116]:
country_codes = pd.read_csv(country_codes_path)
print(country_codes.head())
len(country_codes)



             name alpha-2 alpha-3  country-code     iso_3166-2   region  \
0     Afghanistan      AF     AFG             4  ISO 3166-2:AF     Asia   
1   Åland Islands      AX     ALA           248  ISO 3166-2:AX   Europe   
2         Albania      AL     ALB             8  ISO 3166-2:AL   Europe   
3         Algeria      DZ     DZA            12  ISO 3166-2:DZ   Africa   
4  American Samoa      AS     ASM            16  ISO 3166-2:AS  Oceania   

        sub-region intermediate-region  region-code  sub-region-code  \
0    Southern Asia                 NaN        142.0             34.0   
1  Northern Europe                 NaN        150.0            154.0   
2  Southern Europe                 NaN        150.0             39.0   
3  Northern Africa                 NaN          2.0             15.0   
4        Polynesia                 NaN          9.0             61.0   

   intermediate-region-code  
0                       NaN  
1                       NaN  
2                       Na

249

In [117]:
# rename some country names:
country_codes.replace(to_replace="Türkiye", value="Turkey",inplace=True)
country_codes.replace(to_replace="United Kingdom of Great Britain and Northern Ireland", value="United Kingdom",inplace=True)
country_codes.replace(to_replace="United States of America", value="United States",inplace=True)
country_codes.replace(to_replace="Russian Federation", value="Russia",inplace=True)
country_codes.replace(to_replace="Netherlands, Kingdom of the", value="Netherlands",inplace=True)
country_codes.replace(to_replace="Czechia", value="Czech Republic",inplace=True)
country_codes.replace(to_replace="Iran, Islamic Republic of", value="Iran",inplace=True)
country_codes.replace(to_replace="Korea, Democratic People's Republic of", value="North Korea",inplace=True)
country_codes.replace(to_replace="Korea, Republic of", value="South Korea",inplace=True)
country_codes.replace(to_replace="Viet Nam", value="Vietnam",inplace=True)
country_codes.replace(to_replace="Taiwan, Province of China", value="Taiwan",inplace=True)
country_codes.replace(to_replace="Tanzania, United Republic of", value="Tanzania",inplace=True)
country_codes.replace(to_replace="Côte d'Ivoire", value="Ivory Coast",inplace=True)
country_codes.replace(to_replace="Syrian Arab Republic", value="Syria",inplace=True)
country_codes.replace(to_replace="Congo, Democratic Republic of the", value="Democratic Republic of the Congo",inplace=True)
country_codes.replace(to_replace="Lao People's Democratic Republic", value="Laos",inplace=True)
country_codes.replace(to_replace="Bolivia, Plurinational State of", value="Bolivia",inplace=True)
country_codes.replace(to_replace="Falkland Islands (Malvinas)", value="Falkland Islands",inplace=True)
country_codes.replace(to_replace="Venezuela, Bolivarian Republic of", value="Venezuela",inplace=True)
country_codes.replace(to_replace="Moldova, Republic of", value="Moldova",inplace=True)
country_codes.replace(to_replace="Brunei Darussalam", value="Brunei",inplace=True)
country_codes.replace(to_replace="Palestine, State of", value="Palestine",inplace=True)



In [118]:
col_with_country_codes = col.merge(country_codes, left_on='Country', right_on='name', how='left')


In [119]:
# there should not be any null values
nan_rows = col_with_country_codes[col_with_country_codes['alpha-2'].isnull()]
#Kosovo does not have ISO code
#Namibia alpha-2 code for some reason does not work properly
nan_rows

,Rank,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Year,Country,City,...,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
477,478,31.5,9.3,20.9,27.4,24.8,35.4,2023-06-01,Kosovo,Pristina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
483,484,30.8,11.3,21.5,27.2,32.8,39.2,2023-06-01,Namibia,Windhoek,...,NaN,NAM,516.0,ISO 3166-2:NA,Africa,Sub-Saharan Africa,Southern Africa,2.0,202.0,18.0
835,287,25.4,7.3,17.3,26.2,22.0,57.3,2025-01-01,Kosovo,Pristina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1192,317,51.4,19.7,35.3,43.5,45.4,58.6,2015-06-01,Namibia,Windhoek,...,NaN,NAM,516.0,ISO 3166-2:NA,Africa,Sub-Saharan Africa,Southern Africa,2.0,202.0,18.0
1350,475,30.1,8.8,19.2,24.7,25.0,65.4,2015-06-01,Kosovo,Pristina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1657,268,63.9,30.4,47.8,59.7,51.5,38.6,2014-06-01,Namibia,Windhoek,...,NaN,NAM,516.0,ISO 3166-2:NA,Africa,Sub-Saharan Africa,Southern Africa,2.0,202.0,18.0
1765,376,47.2,18.3,33.3,36.0,34.2,27.3,2014-06-01,Kosovo,Pristina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2316,485,26.3,8.3,17.7,23.5,19.6,39.9,2022-06-01,Kosovo,Pristina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2616,275,40.5,20.7,31.3,36.8,35.8,45.3,2016-06-01,Namibia,Windhoek,...,NaN,NAM,516.0,ISO 3166-2:NA,Africa,Sub-Saharan Africa,Southern Africa,2.0,202.0,18.0
2988,276,44.7,19.2,32.5,41.4,39.4,61.3,2019-01-01,Namibia,Windhoek,...,NaN,NAM,516.0,ISO 3166-2:NA,Africa,Sub-Saharan Africa,Southern Africa,2.0,202.0,18.0


In [120]:
col_with_country_codes = col_with_country_codes[[
    'Rank',
    'Cost of Living Index',
    'Rent Index',
    'Cost of Living Plus Rent Index',
    'Groceries Index',
    'Restaurant Price Index',
    'Local Purchasing Power Index',
    'Year',
    'Country',
    'City',
    'alpha-2',
    'alpha-3',
    'region',
    'sub-region']]

In [124]:
col_with_country_codes.head()

,Rank,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Year,Country,City,alpha-2,alpha-3,region,sub-region
0,1,140.4,93.6,118.1,144.1,141.9,74.3,2023-06-01,Bermuda,Hamilton,BM,BMU,Americas,Northern America
1,2,130.0,46.4,90.2,137.4,121.7,113.2,2023-06-01,Switzerland,Basel,CH,CHE,Europe,Western Europe
2,3,128.5,69.1,100.2,120.4,129.7,104.0,2023-06-01,Switzerland,Zurich,CH,CHE,Europe,Western Europe
3,4,120.6,53.4,88.6,116.1,114.7,99.4,2023-06-01,Switzerland,Lausanne,CH,CHE,Europe,Western Europe
4,5,119.1,66.7,94.1,112.6,120.7,121.4,2023-06-01,Switzerland,Zug,CH,CHE,Europe,Western Europe


In [122]:
col_with_country_codes.to_csv(os.path.join(cwd,"cost_of_living_cities.csv"),index=False, header=[
    'rank',
    'cost_of_living_index',
    'rent_index',
    'cost_of_living_plus_rent_index',
    'groceries_index',
    'restaurant_price_index',
    'local_purchasing_power_index',
    'year',
    'country',
    'city',
    'alpha_2',
    'alpha_3',
    'region',
    'sub_region'])